In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr’

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/data/home/marmot/camtrap/PyCharm/CameraTraps-benchmark')

In [3]:
import json
from collections import OrderedDict
from copy import deepcopy
import os
from copy import deepcopy

from PIL import Image

from data_management.cct_json_utils import CameraTrapJsonUtils
from visualization.visualization_utils import plot_stacked_bar_chart, render_db_bounding_boxes, resize_image

## Caltech bboxes cleanup

The CCT bbox database doesn't have categories in the conventional format (needs to be the detection categories, not species, and "empty" is implied by not having a bbox entry), and the 20190409 version did not have Sara's original bboxes. 

In [4]:
with open('/beaver_disk/camtrap/caltech/original/CaltechCameraTrapsBboxes.json') as f:
    original_boxes = json.load(f)

In [17]:
len(original_boxes['images'])
original_boxes['images'][1000]

63102

{'date_captured': '2014-05-19 07:44:53',
 'file_name': '5995adaf-23d2-11e8-a6a3-ec086b02610b.jpg',
 'frame_num': 3,
 'height': 1494,
 'id': '5995adaf-23d2-11e8-a6a3-ec086b02610b',
 'location': 37,
 'rights_holder': 'Justin Brown',
 'seq_id': '70181c0a-5567-11e8-b59c-dca9047ef277',
 'seq_num_frames': 3,
 'width': 2048}

In [6]:
images_w_annotations = set()
for a in original_boxes['annotations']:
    images_w_annotations.add(a['image_id'])
    
len(images_w_annotations)

63102

In [22]:
# the 'location' field needs to be a string
for i in original_boxes['images']:
    i['location'] = str(i['location'])

Only images with annotations are in this DB - good.

I don't think the cars have bounding boxes yet...

In [7]:
original_boxes['categories']

[{'id': 6, 'name': 'bobcat'},
 {'id': 1, 'name': 'opossum'},
 {'id': 30, 'name': 'empty'},
 {'id': 9, 'name': 'coyote'},
 {'id': 3, 'name': 'raccoon'},
 {'id': 11, 'name': 'bird'},
 {'id': 8, 'name': 'dog'},
 {'id': 16, 'name': 'cat'},
 {'id': 5, 'name': 'squirrel'},
 {'id': 10, 'name': 'rabbit'},
 {'id': 7, 'name': 'skunk'},
 {'id': 14, 'name': 'lizard'},
 {'id': 99, 'name': 'rodent'},
 {'id': 21, 'name': 'badger'},
 {'id': 34, 'name': 'deer'},
 {'id': 37, 'name': 'cow'},
 {'id': 33, 'name': 'car'},
 {'id': 51, 'name': 'fox'},
 {'id': 39, 'name': 'pig'},
 {'id': 40, 'name': 'mountain_lion'},
 {'id': 66, 'name': 'bat'},
 {'id': 97, 'name': 'insect'}]

In [8]:
len(original_boxes['annotations'])
original_boxes['annotations'][1000]

66406

{'bbox': [716.8, 629.76, 261.12, 215.03999999999996],
 'category_id': 9,
 'id': '2a5d2808-cbf1-11e8-819c-970a9450cdbc',
 'image_id': '59f79a18-23d2-11e8-a6a3-ec086b02610b'}

In [9]:
new_categories = [
  {
   "name": "animal",
   "id": 1
  },
  {
   "name": "person",
   "id": 2
  },
  {
   "name": "group",
   "id": 3
  },
  {
   "name": "vehicle",
   "id": 4
  }
]

In [10]:
car_annos = []
empty_annos = []

for a in original_boxes['annotations']:
    if a['category_id'] == 33:
        car_annos.append(a)
    if a['category_id'] == 30:
        empty_annos.append(a)

In [11]:
len(car_annos)
len(empty_annos)

2615

0

In [28]:
cct_dir = '/beaver_disk/camtrap/caltech/cct_images'

for a in car_annos[100:120]:
    image = Image.open(os.path.join(cct_dir, a['image_id'] + '.jpg'))
    render_db_bounding_boxes([a['bbox']] , [1], image, original_size=None, label_map=None, thickness=4)
    print(a['bbox'])
    #image

[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2560, 1920]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2048, 1536]
[0, 0, 2560, 1920]


In [12]:
new_annos = []

for a in original_boxes['annotations']:
    original_cat = a['category_id']
    new_anno = deepcopy(a)
    
    if original_cat == 33:  # cars
        new_anno['category_id'] = 4
    else:
        new_anno['category_id'] = 1
    new_annos.append(new_anno)
len(new_annos)

66406

In [13]:
new_annos[100:102]

[{'bbox': [883.2, 612.693359375, 320.8533593750001, 358.4],
  'category_id': 1,
  'id': '2a551c6c-cbf1-11e8-819c-970a9450cdbc',
  'image_id': '59421138-23d2-11e8-a6a3-ec086b02610b'},
 {'bbox': [1448.5743761062913,
   747.0000267028745,
   484.5405578612941,
   633.4358334541191],
  'category_id': 1,
  'id': '2a551d5c-cbf1-11e8-819c-970a9450cdbc',
  'image_id': '59ffba4c-23d2-11e8-a6a3-ec086b02610b'}]

In [14]:
original_boxes['info']

{'contributor': 'Sara Beery',
 'date_created': '2018-10-09 18:51:47.161213',
 'description': 'Bounding box annotations for 63,102 images from Caltech Camera Traps, covering all classes and locations.  Contains all annotations for CCT - 20, the 20-location dataset used in the ECCV18 paper "Recognition in Terra Incognita," as well as additional annotations collected by MS AI for Earth',
 'version': 'Caltech Camera Traps - Bboxes',
 'year': 2018}

In [16]:
new_info = deepcopy(original_boxes['info'])
new_info['version'] = '20190904'
new_info['description'] = original_boxes['info']['description'] + '. Version 20190904 has the categories normed to the four detector categories.'
new_info

{'contributor': 'Sara Beery',
 'date_created': '2018-10-09 18:51:47.161213',
 'description': 'Bounding box annotations for 63,102 images from Caltech Camera Traps, covering all classes and locations.  Contains all annotations for CCT - 20, the 20-location dataset used in the ECCV18 paper "Recognition in Terra Incognita," as well as additional annotations collected by MS AI for Earth. Version 20190904 has the categories normed to the four detector categories.',
 'version': '20190904',
 'year': 2018}

In [23]:
new_bboxes_db = {
    'info': new_info,
    'categories': new_categories,
    'images': original_boxes['images'],
    'annotations': new_annos
}
new_bboxes_db = CameraTrapJsonUtils.order_db_keys(new_bboxes_db)

In [24]:
with open('/beaver_disk/camtrap/caltech/original/caltech_bboxes_20190904.json', 'w') as f:
    json.dump(new_bboxes_db, f, indent=1)